In [28]:
import pandas as pd
import numpy as np

In [29]:
pd.set_option('display.max_columns', None)

# 00. Loading data

In [30]:
PATH = '../data/'

## Score data
This data is the one we will use as input to our prediction

In [31]:
score_historical = pd.read_csv(PATH + 'input/' + 'spreadspoke_scores.csv')
score_historical = score_historical[[
    'schedule_date',
    'schedule_season',
    'schedule_week',
    'schedule_playoff',
    'team_home',
    'score_home',
    'score_away',
    'team_away',
    'stadium_neutral'
]]
score_historical.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral
0,9/2/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,False
1,9/3/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,False
2,9/4/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,False
3,9/9/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,False
4,9/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,False


Translate team name for pattern.

In [32]:
set([*score_historical['team_home']] + [*score_historical['team_away']])

{'Arizona Cardinals',
 'Atlanta Falcons',
 'Baltimore Colts',
 'Baltimore Ravens',
 'Boston Patriots',
 'Buffalo Bills',
 'Carolina Panthers',
 'Chicago Bears',
 'Cincinnati Bengals',
 'Cleveland Browns',
 'Dallas Cowboys',
 'Denver Broncos',
 'Detroit Lions',
 'Green Bay Packers',
 'Houston Oilers',
 'Houston Texans',
 'Indianapolis Colts',
 'Jacksonville Jaguars',
 'Kansas City Chiefs',
 'Las Vegas Raiders',
 'Los Angeles Chargers',
 'Los Angeles Raiders',
 'Los Angeles Rams',
 'Miami Dolphins',
 'Minnesota Vikings',
 'New England Patriots',
 'New Orleans Saints',
 'New York Giants',
 'New York Jets',
 'Oakland Raiders',
 'Philadelphia Eagles',
 'Phoenix Cardinals',
 'Pittsburgh Steelers',
 'San Diego Chargers',
 'San Francisco 49ers',
 'Seattle Seahawks',
 'St. Louis Cardinals',
 'St. Louis Rams',
 'Tampa Bay Buccaneers',
 'Tennessee Oilers',
 'Tennessee Titans',
 'Washington Commanders',
 'Washington Football Team',
 'Washington Redskins'}

In [33]:
# Change Commanders name to get historic
commanders_name = [
    'Washington Commanders',
    'Washington Football Team',
    'Washington Redskins'
]

def change_commanders_name(team_name, commanders_name):
  if team_name in commanders_name:
    return 'Washington Commanders'
  else:
    return team_name

score_historical['team_home'] = score_historical.apply(lambda row: change_commanders_name(row['team_home'], commanders_name), axis=1)
score_historical['team_away'] = score_historical.apply(lambda row: change_commanders_name(row['team_away'], commanders_name), axis=1)

In [34]:
score_historical['team_home'] = score_historical['team_home'].str.split(' ').str[-1]
score_historical['team_away'] = score_historical['team_away'].str.split(' ').str[-1]

Creating target column, this will be used to teach our model.

In [ ]:
def is_home_winner(row):
    if row['score_home'] > row['score_away']:
        return 1
    else:
        return 0
    
score_historical['is_home_winner'] = score_historical.apply(is_home_winner, axis=1)

This is our history to enrich our input. Just like the input, but with scores.

In [36]:
score_historical.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,is_home_winner
0,9/2/1966,1966,1,False,Dolphins,14,23,Raiders,False,0
1,9/3/1966,1966,1,False,Oilers,45,7,Broncos,False,1
2,9/4/1966,1966,1,False,Chargers,27,7,Bills,False,1
3,9/9/1966,1966,2,False,Dolphins,14,19,Jets,False,0
4,9/10/1966,1966,1,False,Packers,24,3,Colts,False,1


## Stats Data
This data will enrich our input to improve our prediction.

In [37]:
# Iter over a list so we don't need to call each df individually
data_list = [
  'defense_downs',
  'defense_fumbles',
  'defense_interceptions',
  'defense_passing',
  'defense_receiving',
  'defense_rushing',
  'defense_scoring',
  'defense_tackles',
  'offense_downs',
  'offense_passing',
  'offense_receiving',
  'offense_rushing',
  'offense_scoring',
  'special-teams_field-goals',
  'special-teams_kickoff-returns',
  'special-teams_kickoffs',
  'special-teams_punt-returns',
  'special-teams_punting',
  'special-teams_scoring',
]

In [38]:
dataframe_dict = {}
for item in data_list:
  data = pd.read_csv(PATH + 'input/' + f'{item}.csv')
  dataframe_dict[item] = data

In [39]:
# Unpack dict into each of the dataframe variables
(
  defense_downs,
  defense_fumbles,
  defense_interceptions,
  defense_passing,
  defense_receiving,
  defense_rushing,
  defense_scoring,
  defense_tackles,
  offense_downs,
  offense_passing,
  offense_receiving,
  offense_rushing,
  offense_scoring,
  special_teams_field_goals,
  special_teams_kickoff_returns,
  special_teams_kickoffs,
  special_teams_punt_returns,
  special_teams_punting,
  special_teams_scoring
) = tuple(dataframe_dict.values())

In [40]:
# Checking result
offense_rushing.head()

,Team,Att,Rush Yds,YPC,TD,20+,40+,Lng,Rush 1st,Rush 1st%,Rush FUM,year
0,Chiefs,556,2627,4.7,13,15,4,80T,129,23.2,7,2010
1,Jets,534,2374,4.4,14,11,2,53,118,22.1,11,2010
2,Jaguars,512,2395,4.7,14,14,1,74,148,28.9,8,2010
3,Raiders,504,2494,5.0,19,27,6,71,113,22.4,9,2010
4,Falcons,497,1891,3.8,14,12,1,55,111,22.3,4,2010


In [41]:
def get_df_name(df):
  name =[x for x in globals() if globals()[x] is df][0]
  return name

def change_dataframes_columns(dataframes_list):
  keep_names = ['year', 'Team']
  for df in dataframes_list:
    df_name = get_df_name(df)
    new_columns = []
    for column in df.columns:
      if not column in (keep_names):
        new_name = f'{df_name}_{column}'
      else:
        new_name = column
      new_columns.append(new_name)
    df.columns = new_columns

def merge_dataframes(dataframes_list):
  new_dataframe = dataframes_list[0].copy()
  for df in dataframes_list[1:]:
    new_dataframe = new_dataframe.merge(df,
                                        on=['year', 'Team'],
                                        how='left')
  return new_dataframe

In [42]:
defense_dfs = [
  defense_downs,
  defense_fumbles,
  defense_interceptions,
  defense_passing,
  defense_receiving,
  defense_rushing,
  defense_scoring,
  defense_tackles,
]

change_dataframes_columns(defense_dfs)
defense_dataframe = merge_dataframes(defense_dfs)

In [43]:
offense_dfs = [
  offense_downs,
  offense_passing,
  offense_receiving,
  offense_rushing,
  offense_scoring,
]

change_dataframes_columns(offense_dfs)
offense_dataframe = merge_dataframes(offense_dfs)

In [44]:
special_teams_dfs = [
  special_teams_field_goals,
  special_teams_kickoff_returns,
  special_teams_kickoffs,
  special_teams_punt_returns,
  special_teams_punting,
  special_teams_scoring
]

change_dataframes_columns(special_teams_dfs)
special_teams_dataframe = merge_dataframes(special_teams_dfs)

In [45]:
stats_dataframe = defense_dataframe.merge(offense_dataframe,
                                          on=['year', 'Team'],
                                          how='left')

stats_dataframe = stats_dataframe.merge(special_teams_dataframe,
                                        on=['year', 'Team'],
                                        how='left')

Fixing team name for pattern.

In [46]:
stats_dataframe['Team'].sort_values().unique()

array(['49ers', 'Bears', 'Bengals', 'Bills', 'Broncos', 'Browns',
       'Buccaneers', 'Cardinals', 'Chargers', 'Chiefs', 'Colts',
       'Commanders', 'Cowboys', 'Dolphins', 'Eagles', 'Falcons',
       'FootballTeam', 'Giants', 'Jaguars', 'Jets', 'Lions', 'Niners',
       'Packers', 'Panthers', 'Patriots', 'Raiders', 'Rams', 'Ravens',
       'Redskins', 'Saints', 'Seahawks', 'Steelers', 'Texans', 'Titans',
       'Vikings'], dtype=object)

In [47]:
teams_to_translate = {
    'Niners' : '49ers',
    'Redskins' : 'Commanders',
    'FootballTeam' : 'Commanders'
}

def translate_team_names(team_name, teams_to_translate):
  if team_name in teams_to_translate.keys():
    return teams_to_translate.get(team_name)
  else:
    return team_name

stats_dataframe['Team'] = stats_dataframe.apply(lambda row: translate_team_names(row['Team'], teams_to_translate), axis=1)

Lagging year so we can use last year as stats for current predicting data.

In [48]:
stats_dataframe['year'] = stats_dataframe['year'] + 1 # the last year stats refers to this years game
stats_dataframe.rename(columns={'year' : 'stats_year'}, inplace=True)

This is our stats data so far.

In [49]:
stats_dataframe.head()

,Team,defense_downs_3rd Att,defense_downs_3rd Md,defense_downs_4th Att,defense_downs_4th Md,defense_downs_Rec 1st,defense_downs_Rec 1st%,defense_downs_Rush 1st,defense_downs_Rush 1st%,defense_downs_Scrm Plys,stats_year,defense_fumbles_FF,defense_fumbles_FR,defense_fumbles_FR TD,defense_fumbles_Rec FUM,defense_fumbles_Rush FUM,defense_interceptions_INT,defense_interceptions_INT TD,defense_interceptions_INT Yds,defense_interceptions_Lng,defense_passing_Att,defense_passing_Cmp,defense_passing_Cmp %,defense_passing_Yds/Att,defense_passing_Yds,defense_passing_TD,defense_passing_INT,defense_passing_Rate,defense_passing_1st,defense_passing_1st%,defense_passing_20+,defense_passing_40+,defense_passing_Lng,defense_passing_Sck,defense_receiving_Rec,defense_receiving_Yds,defense_receiving_Yds/Rec,defense_receiving_TD,defense_receiving_20+,defense_receiving_40+,defense_receiving_Lng,defense_receiving_Rec 1st,defense_receiving_Rec 1st%,defense_receiving_Rec FUM,defense_receiving_PDef,defense_rushing_Att,defense_rushing_Rush Yds,defense_rushing_YPC,defense_rushing_TD,defense_rushing_20+,defense_rushing_40+,defense_rushing_Lng,defense_rushing_Rush 1st,defense_rushing_Rush 1st%,defense_rushing_Rush FUM,defense_scoring_FR TD,defense_scoring_SFTY,defense_scoring_INT TD,defense_tackles_Sck,defense_tackles_Comb,defense_tackles_Asst,defense_tackles_Solo,offense_downs_3rd Att,offense_downs_3rd Md,offense_downs_4th Att,offense_downs_4th Md,offense_downs_Rec 1st,offense_downs_Rec 1st%,offense_downs_Rush 1st,offense_downs_Rush 1st%,offense_downs_Scrm Plys,offense_passing_Att,offense_passing_Cmp,offense_passing_Cmp %,offense_passing_Yds/Att,offense_passing_Pass Yds,offense_passing_TD,offense_passing_INT,offense_passing_Rate,offense_passing_1st,offense_passing_1st%,offense_passing_20+,offense_passing_40+,offense_passing_Lng,offense_passing_Sck,offense_passing_SckY,offense_receiving_Rec,offense_receiving_Yds,offense_receiving_Yds/Rec,offense_receiving_TD,offense_receiving_20+,offense_receiving_40+,offense_receiving_Lng,offense_receiving_Rec 1st,offense_receiving_Rec 1st%,offense_receiving_Rec FUM,offense_rushing_Att,offense_rushing_Rush Yds,offense_rushing_YPC,offense_rushing_TD,offense_rushing_20+,offense_rushing_40+,offense_rushing_Lng,offense_rushing_Rush 1st,offense_rushing_Rush 1st%,offense_rushing_Rush FUM,offense_scoring_Rsh TD,offense_scoring_Rec TD,offense_scoring_Tot TD,offense_scoring_2-PT,special_teams_field_goals_FGM,special_teams_field_goals_Att,special_teams_field_goals_FG %,special_teams_field_goals_1-19 > A-M,special_teams_field_goals_20-29 > A-M,special_teams_field_goals_30-39 > A-M,special_teams_field_goals_40-49 > A-M,special_teams_field_goals_50-59 > A-M,special_teams_field_goals_60+ > A-M,special_teams_field_goals_Lng,special_teams_field_goals_FG Blk,special_teams_kickoff_returns_Avg,special_teams_kickoff_returns_Ret,special_teams_kickoff_returns_Yds,special_teams_kickoff_returns_KRet TD,special_teams_kickoff_returns_20+,special_teams_kickoff_returns_40+,special_teams_kickoff_returns_Lng,special_teams_kickoff_returns_FC,special_teams_kickoff_returns_FUM,special_teams_kickoff_returns_FG Blk,special_teams_kickoff_returns_XP Blk,special_teams_kickoffs_KO,special_teams_kickoffs_Yds,special_teams_kickoffs_TB,special_teams_kickoffs_TB %,special_teams_kickoffs_Ret,special_teams_kickoffs_Ret Avg,special_teams_kickoffs_OSK,special_teams_kickoffs_OSK Rec,special_teams_kickoffs_OOB,special_teams_kickoffs_TD,special_teams_punt_returns_Avg,special_teams_punt_returns_Ret,special_teams_punt_returns_Yds,special_teams_punt_returns_PRet T,special_teams_punt_returns_20+,special_teams_punt_returns_40+,special_teams_punt_returns_Lng,special_teams_punt_returns_FC,special_teams_punt_returns_FUM,special_teams_punt_returns_P Blk,special_teams_punting_Att,special_teams_punting_Cmp,special_teams_punting_Cmp %,special_teams_punting_Yds/Att,special_teams_punting_Pass Yds,special_teams_punting_TD,special_teams_punting_INT,special_teams_punting_Rate,special_teams_

### Improving stats dataframe

In [50]:
# Let's make some functions to clean our stats dataframe
# Cleaning % data
def clean_percent_data(df):
  for column in df.columns:
    if '%' in column:
      df[column] = df[column] / 100

# Remove special characters from columns
def remove_special_char_columns(df):
  for column in df.columns:
    new_name = column.lower().replace(' ','_').replace('%', '_perc')
    df.rename(columns={
        column : new_name
    }, inplace=True)

# Transform turnover in negative data
def negative_turnover_number(turnover_columns, df):
  for column in turnover_columns:
    new_column = []
    for item in df[column]:
      if 'T' in item:
        new_value = item.replace('T', '')
        new_value = int(new_value)
        new_value = new_value * -1
      else:
        new_value = int(item)
      new_column.append(new_value)
    df[column] = new_column

In [51]:
turnover_columns = [
    'defense_passing_lng',
    'defense_receiving_lng',
    'defense_rushing_lng',
    'defense_interceptions_lng',
    'offense_passing_lng',
    'offense_receiving_lng',
    'offense_rushing_lng',
    'special_teams_kickoff_returns_lng',
    'special_teams_punt_returns_lng',
    'special_teams_punting_lng'
]

clean_percent_data(stats_dataframe)
remove_special_char_columns(stats_dataframe)
negative_turnover_number(turnover_columns, stats_dataframe)

In [52]:
# Fix columns with A_M (attemps_made)
a_m_columns = [
    'special_teams_field_goals_1-19_>_a-m',
    'special_teams_field_goals_20-29_>_a-m',
    'special_teams_field_goals_30-39_>_a-m',
    'special_teams_field_goals_40-49_>_a-m',
    'special_teams_field_goals_50-59_>_a-m',
    'special_teams_field_goals_60+_>_a-m',
]

# We will transform it in a percent so we don't need to create a new column for each case
for column in a_m_columns:
  attps_list = stats_dataframe[column].str.split('_').str[0]
  matches_list = stats_dataframe[column].str.split('_').str[1]

  percent_matches = matches_list.astype('int') / attps_list.astype('int')
  percent_matches.fillna(0, inplace=True)
  stats_dataframe[column] = percent_matches
  stats_dataframe.rename(columns={
      column : column.replace('a-m', 'percent_a_m')
  }, inplace=True)

In [53]:
# Removing non important columns (trash from the origin)
columns_to_drop = [
    'data_fgm',
    'data_fg__perc',
    'data_xpm',
    'data_xp_pct',
    'data_kret_td',
    'data_pret_t'
]

stats_dataframe.drop(columns_to_drop,
                     axis=1,
                     inplace=True)

In [54]:
stats_dataframe.head()

,team,defense_downs_3rd_att,defense_downs_3rd_md,defense_downs_4th_att,defense_downs_4th_md,defense_downs_rec_1st,defense_downs_rec_1st_perc,defense_downs_rush_1st,defense_downs_rush_1st_perc,defense_downs_scrm_plys,stats_year,defense_fumbles_ff,defense_fumbles_fr,defense_fumbles_fr_td,defense_fumbles_rec_fum,defense_fumbles_rush_fum,defense_interceptions_int,defense_interceptions_int_td,defense_interceptions_int_yds,defense_interceptions_lng,defense_passing_att,defense_passing_cmp,defense_passing_cmp__perc,defense_passing_yds/att,defense_passing_yds,defense_passing_td,defense_passing_int,defense_passing_rate,defense_passing_1st,defense_passing_1st_perc,defense_passing_20+,defense_passing_40+,defense_passing_lng,defense_passing_sck,defense_receiving_rec,defense_receiving_yds,defense_receiving_yds/rec,defense_receiving_td,defense_receiving_20+,defense_receiving_40+,defense_receiving_lng,defense_receiving_rec_1st,defense_receiving_rec_1st_perc,defense_receiving_rec_fum,defense_receiving_pdef,defense_rushing_att,defense_rushing_rush_yds,defense_rushing_ypc,defense_rushing_td,defense_rushing_20+,defense_rushing_40+,defense_rushing_lng,defense_rushing_rush_1st,defense_rushing_rush_1st_perc,defense_rushing_rush_fum,defense_scoring_fr_td,defense_scoring_sfty,defense_scoring_int_td,defense_tackles_sck,defense_tackles_comb,defense_tackles_asst,defense_tackles_solo,offense_downs_3rd_att,offense_downs_3rd_md,offense_downs_4th_att,offense_downs_4th_md,offense_downs_rec_1st,offense_downs_rec_1st_perc,offense_downs_rush_1st,offense_downs_rush_1st_perc,offense_downs_scrm_plys,offense_passing_att,offense_passing_cmp,offense_passing_cmp__perc,offense_passing_yds/att,offense_passing_pass_yds,offense_passing_td,offense_passing_int,offense_passing_rate,offense_passing_1st,offense_passing_1st_perc,offense_passing_20+,offense_passing_40+,offense_passing_lng,offense_passing_sck,offense_passing_scky,offense_receiving_rec,offense_receiving_yds,offense_receiving_yds/rec,offense_receiving_td,offense_receiving_20+,offense_receiving_40+,offense_receiving_lng,offense_receiving_rec_1st,offense_receiving_rec_1st_perc,offense_receiving_rec_fum,offense_rushing_att,offense_rushing_rush_yds,offense_rushing_ypc,offense_rushing_td,offense_rushing_20+,offense_rushing_40+,offense_rushing_lng,offense_rushing_rush_1st,offense_rushing_rush_1st_perc,offense_rushing_rush_fum,offense_scoring_rsh_td,offense_scoring_rec_td,offense_scoring_tot_td,offense_scoring_2-pt,special_teams_field_goals_fgm,special_teams_field_goals_att,special_teams_field_goals_fg__perc,special_teams_field_goals_1-19_>_percent_a_m,special_teams_field_goals_20-29_>_percent_a_m,special_teams_field_goals_30-39_>_percent_a_m,special_teams_field_goals_40-49_>_percent_a_m,special_teams_field_goals_50-59_>_percent_a_m,special_teams_field_goals_60+_>_percent_a_m,special_teams_field_goals_lng,special_teams_field_goals_fg_blk,special_teams_kickoff_returns_avg,special_teams_kickoff_returns_ret,special_teams_kickoff_returns_yds,special_teams_kickoff_returns_kret_td,special_teams_kickoff_returns_20+,special_teams_kickoff_returns_40+,special_teams_kickoff_returns_lng,special_teams_kickoff_returns_fc,special_teams_kickoff_returns_fum,special_teams_kickoff_returns_fg_blk,special_teams_kickoff_returns_xp_blk,special_teams_kickoffs_ko,special_teams_kickoffs_yds,special_teams_kickoffs_tb,special_teams_kickoffs_tb__perc,special_teams_kickoffs_ret,special_teams_kickoffs_ret_avg,special_teams_kickoffs_osk,special_teams_kickoffs_osk_rec,special_teams_kickoffs_oob,special_teams_kickoffs_td,special_teams_punt_returns_avg,special_teams_punt_returns_ret,special_teams_punt_returns_yds,special_teams_punt_returns_pret_t,special_teams_punt_returns_20+,special_teams_punt_returns_40+,special_teams_punt_returns_lng,special_teams_punt_returns_fc,special_teams_punt_returns_fum,special_teams_punt_returns_p_blk,special_teams_punting_att,special_teams_punting_cmp,special_teams_punting_cmp__perc,special_teams_punting_yds/att,special_teams_p

# Creating train, validation and test dataframe
Now we need to create a class to join stats with our input and create new features if needed. The pipeline will do as follow:
1. We read a dataframe with the data we want to predict;
2. We append that data to the stats dataframe;
3. We create a few more features historic based;
4. We sent this data as input (one line of data).

In [55]:
train_data = score_historical[score_historical['schedule_season'] < 2024]
test_data = score_historical[score_historical['schedule_season'] == 2024]

In [71]:
target = 'is_home_winner'
X_train = train_data.drop(target, axis=1) # What we will use to create our algorithm
y_train = train_data[target]
X_train.drop(columns=['score_home', 'score_away'], inplace=True)

X_test = test_data.drop(target, axis=1) # What we will use as input
y_test = test_data[target]
X_test.drop(columns=['score_home', 'score_away'], inplace=True)

In [105]:
class EnrichInput:
    def __init__(self, historical_df, stats_df, X):
        self.historical_df = historical_df
        self.stats_df = stats_df
        self.X = X
        self.historic = {}

    def calculate_historic(self, row):
        team_1, team_2 = sorted([row['team_home'], row['team_away']])
        key = (team_1, team_2)
        if key not in self.historic:
            self.historic[key] = {'victories_team_1': 0, 'victories_team_2': 0, 'matches': 0}
        else:
            if row['is_home_winner']:
                if row['team_home'] == team_1:
                    self.historic[key]['victories_team_1'] += 1
                else:
                    self.historic[key]['victories_team_2'] += 1
            else:
                if row['team_home'] == team_1:
                    self.historic[key]['victories_team_2'] += 1
                else:
                    self.historic[key]['victories_team_1'] += 1

        self.historic[key]['matches'] += 1
        if row['team_home'] == team_1:
            return self.historic[key]['victories_team_1'] / self.historic[key]['matches'], self.historic[key]['matches']
        else:
            return self.historic[key]['victories_team_2'] / self.historic[key]['matches'], self.historic[key]['matches']

    def create_away_historic(self, row):
        if row['matches'] > 1:
            return 1 - row['home_win_historic_confront']
        else:
            return 0

    def get_game_history(self):
        _historical_df = self.historical_df
        _historical_df['home_win_historic_confront', 'matches'] = _historical_df.apply(self.calculate_historic, axis=1).to_list()
        _historical_df['away_win_historic_confront'] = _historical_df.apply(self.create_away_historic, axis=1)
        _historical_df = _historical_df[[
            'schedule_date',
            'team_home',
            'team_away',
            'home_win_historic_confront',
            'away_win_historic_confront',
            'matches',
        ]]
        X_history = self.X.merge(
            _historical_df,
            on=['schedule_date', 'team_home', 'team_away'],
            how='left'
        )
        return X_history

In [106]:
EnrichInput(score_historical, stats_dataframe, X_train).get_game_history()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,team_away,stadium_neutral,home_win_historic_confront,away_win_historic_confront,matches
0,9/2/1966,1966,1,False,Dolphins,Raiders,False,0.000000,0.000000,1.0
1,9/3/1966,1966,1,False,Oilers,Broncos,False,0.000000,0.000000,1.0
2,9/4/1966,1966,1,False,Chargers,Bills,False,0.000000,0.000000,1.0
3,9/9/1966,1966,2,False,Dolphins,Jets,False,0.000000,0.000000,1.0
4,9/10/1966,1966,1,False,Packers,Colts,False,0.000000,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...
13796,1/21/2024,2023,Division,True,Bills,Chiefs,False,0.488372,0.511628,43.0
13797,1/21/2024,2023,Division,True,Lions,Buccaneers,False,0.524590,0.475410,61.0
13798,1/28/2024,2023,Conference,True,Ravens,Chiefs,False,0.384615,0.615385,13.0
13799,1/28/2024,2023,Conference,True,49ers,Lions,False,0.638889,0.361111,36.0
